In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

class VGG16_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:

      train_formats = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
      test_formats= ImageDataGenerator(rescale=1./255)
      self.train_data = train_formats.flow_from_directory(training_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.test_data =test_formats.flow_from_directory(testing_data,
                                      target_size=(224,224),batch_size=20,class_mode='categorical',shuffle=False)
      self.valid_data = train_formats.flow_from_directory(validation_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')

      self.labels=os.listdir(training_data)
      #print(self.train_data)

    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model(self):
    try:
      image_size=[224,224]
      vgg16 = VGG16(input_shape=image_size + [3] , weights='imagenet',include_top=False)
      for layers in vgg16.layers:  # VGG16.layers:
        layers.trainable=False
      x =Flatten()(vgg16.output)
      self.predict = Dense(34,activation='softmax')(x)
      self.model = Model(inputs = vgg16.inputs,outputs=self.predict) # VGG16
      print(self.model.summary())
    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


  def training_epoch(self):
    try:
      self.model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
      self.model.fit(self.train_data,epochs=10,
                     validation_data=self.valid_data)
      self.model.save("/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ vgg16_outputs/vgg16_model.h5")
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_evaluation(self):
    try:
      output_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ vgg16_outputs"
      model_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ vgg16_outputs/vgg16_model.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "vgg16_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)
    except Exception as e:
      er_type,er_msg,line_no = sys.exc_info()
      print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')





if __name__=="__main__":
  try:
    training_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/training_data_150"
    testing_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/testing_data_20"
    validation_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/validation_data_30"
    obj=VGG16_MODEL(training_data,testing_data,validation_data)
    obj.model()
  except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

Found 5094 images belonging to 34 classes.
Found 680 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 34)                  │         853,026 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 15,567,714 (59.39 MB)

 Trainable params: 853,026 (3.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

None


In [ ]:
#obj.training_epoch()

Epoch 1/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 5412s 11s/step - accuracy: 0.2347 - loss: 4.1153 - val_accuracy: 0.4676 - val_loss: 2.7698
Epoch 2/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 143s 280ms/step - accuracy: 0.5881 - loss: 1.9767 - val_accuracy: 0.5275 - val_loss: 3.0297
Epoch 3/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 161s 315ms/step - accuracy: 0.6898 - loss: 1.4680 - val_accuracy: 0.6020 - val_loss: 2.2737
Epoch 4/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 141s 276ms/step - accuracy: 0.7399 - loss: 1.1043 - val_accuracy: 0.6029 - val_loss: 2.5799
Epoch 5/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 141s 277ms/step - accuracy: 0.7677 - loss: 1.0830 - val_accuracy: 0.5765 - val_loss: 3.1462
Epoch 6/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 140s 275ms/step - accuracy: 0.8137 - loss: 0.8141 - val_accuracy: 0.5990 - val_loss: 2.8680
Epoch 7/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 145s 284ms/step - accuracy: 0.8222 - loss: 0.7707 - val_accuracy: 0.6118 - val_loss: 2.8805
Epoch 8/10
510/510 ━━━━━━━━━━━━━━━━━━━━ 197s 274ms/step - accuracy: 0.8590 - 

In [3]:

obj.model_evaluation()

34/34 ━━━━━━━━━━━━━━━━━━━━ 442s 13s/step
